Script to collect training data, merge results from the inference runs and collect extended test set

In [32]:
from transformers import pipeline
import csv
import pandas as pd
from IPython.display import Image
from PIL import Image
import requests
from io import BytesIO
import time


df = pd.read_csv("full_data_files/merged_df.csv")
#get column "image 1" from df
image_urls = df["image 1"].unique()
image_urls = [url for url in image_urls if url != 'N']
#retain only the first 1000 image_urls
image_urls = image_urls[:1000]
print(len(image_urls))
len(df["image 1"].unique())





C:\Users\a1224\AppData\Local\Temp\ipykernel_11844\3990344851.py:11: DtypeWarning: Columns (130,134,140,147,152,153) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("full_data_files/merged_df.csv")


1000


14315

Download images

In [14]:


# Specify the user-agent header
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}
k = 0

for url in image_urls:
    time.sleep(1)
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        image_data = response.content
        # Create a PIL Image object from the image data
        image = Image.open(BytesIO(image_data))
        # Save the image to the images folder
        image.save("images/boat_" + str(k) + ".jpg")
        k = k + 1
    else:
        print("Failed to retrieve the image.")
    


Then import images as dataset from folder structure

# Merge all csv files with image features from inference into one

In [16]:
import pandas as pd
df1 = pd.read_csv("imageDF1.csv", sep=";", header=0)["image class1"]
df2 = pd.read_csv("imageDF2.csv", sep=";", header=0)["image class2"]
df3 = pd.read_csv("imageDF3.csv", sep=";", header=0)["image class3"]
df_bright = pd.read_csv("imageDF_brightness.csv", sep=";", header=0)["image brightness"]

In [17]:
# merge all 4 columns into one dataframe
df = pd.concat([df1, df2, df3, df_bright], axis=1)

In [22]:
# make a new column "img_water" which takes value 1 if on of the first 3 columns is "water" and 0 otherwise
df["image water"] = df.apply(lambda x: 1 if x["image class1"] == "water" or x["image class2"] == "water" or x["image class3"] == "water" else 0, axis=1)
df["image land"] = df.apply(lambda x: 1 if x["image class1"] == "land" or x["image class2"] == "land" or x["image class3"] == "land" else 0, axis=1)
#df["both"] = df.apply(lambda x: x["image water"] * x["image land"], axis=1)

Export the complete dataframe to csv

In [24]:
# write the dataframe to a csv file
df[["image water", "image land", "image brightness"]].to_csv("image_features.csv", sep=";", index=False)

# Download image classification extended test set

In [35]:
df = pd.read_csv("imageDF1.csv", sep=";", header=0)
#turn df into strings
df = df.astype(str)

In [36]:


#get column "image 1" from df
image_urls = df["image 2"].unique()
image_urls = [url for url in image_urls if 'https' in url]
#retain only the first 1000 image_urls
image_urls = image_urls[:200]



In [37]:
# Specify the user-agent header
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}
k = 0

for url in image_urls:
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        image_data = response.content
        # Create a PIL Image object from the image data
        image = Image.open(BytesIO(image_data))
        # Save the image to the images folder
        image.save("imageTestSet/water/boat_" + str(k) + ".jpg")
        k = k + 1
    else:
        print("Failed to retrieve the image.")